本节训练一个 sequence-to-sequence 模型，使用pytorch的 
`nn.Transformer <https://pytorch.org/docs/master/nn.html?highlight=nn%20transformer#torch.nn.Transformer>` module.

PyTorch 1.2 基于论文 `Attention is All YouNeed <https://arxiv.org/pdf/1706.03762.pdf>` 实现了一个 Transformer 模型， ``nn.Transformer`` 模块依赖于 attention 机制实现表达输入和输出文本的关系。


Transformer模型的结构图：
![](../docs/transformer_architecture.jpg)


# 定义模型

基于 ``nn.TransformerEncoder`` 模型训练语言模型。

语言模型任务是为句子后跟随单词输出一个似然概率，表征这个单词可能出现的概率。

首先做 embedding，再做 positional encoding, 表征单词位置关系。``nn.TransformerEncoder`` 由多层`nn.TransformerEncoderLayer <https://pytorch.org/docs/master/nn.html?highlight=transformerencoderlayer#torch.nn.TransformerEncoderLayer>`组成，对于语言模型任务，每个未来可能出现的单词都需要 mask 并预测其概率，为了得到实际的预测单词，``nn.TransformerEncoder``模型的输出后需要接一个 log-Softmax 函数。

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5_000) -> None:
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)



class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5) -> None:
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weight()
    
    def init_weight(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)  # torch.triu 让对角线下方区域都变为0
        mask = mask.float().masked_fill(mask==0, float('-inf')).masked_fill(mask==1, float(0.0)) 
        return mask
    
    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output




# 加载数据

模型训练过程使用来自 torchtext 的Wikitext-2数据集。vocab 基于 train 数据集构建。``batchify（）``函数将数据集排列成列，在将数据划分为大小为`batch_size``的批次后，删除所有剩余的标记。

例如，将字母表作为序列（总长度为26），批量大小为4，我们将字母表分成4个长度为6的序列：

\begin{align}\begin{bmatrix}
  \text{A} & \text{B} & \text{C} & \ldots & \text{X} & \text{Y} & \text{Z}
  \end{bmatrix}
  \Rightarrow
  \begin{bmatrix}
  \begin{bmatrix}\text{A} \\ \text{B} \\ \text{C} \\ \text{D} \\ \text{E} \\ \text{F}\end{bmatrix} &
  \begin{bmatrix}\text{G} \\ \text{H} \\ \text{I} \\ \text{J} \\ \text{K} \\ \text{L}\end{bmatrix} &
  \begin{bmatrix}\text{M} \\ \text{N} \\ \text{O} \\ \text{P} \\ \text{Q} \\ \text{R}\end{bmatrix} &
  \begin{bmatrix}\text{S} \\ \text{T} \\ \text{U} \\ \text{V} \\ \text{W} \\ \text{X}\end{bmatrix}
  \end{bmatrix}\end{align}


In [2]:
import os
import torchtext
from torchtext.data.utils import get_tokenizer

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

from torchtext.datasets import IMDB
train_iter, test_iter = IMDB(split=('train', 'test'))



AttributeError: module 'torchtext' has no attribute 'legacy'